<a href="https://colab.research.google.com/github/NikkiBytes/BBx/blob/master/feat3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BBX Quality Analysis 


Quick References-
* google drive: [folder](https://drive.google.com/drive/u/0/folders/1fkNFJ_ODgWX1vBT8KJhAydEU7gEkWV37) 
* renci specific data paths(*bids, fmriprep, etc*): [csv](https://docs.google.com/spreadsheets/d/1dgbbIByS4GKMMRmFmpEug_VTmbopz3-xqhwm4RcLrFU/edit#gid=0)  


## Start

In [ ]:
# establish connection between google colab and google drive with mount
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)



Mounted at /content/gdrive


In [ ]:
# import pacakges 

import os, glob
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from IPython.html.widgets import interactive

#pd.set_option('display.max_rows', 159)
#pd.set_option('display.max_colwidth', -1)

%matplotlib inline 

In [ ]:
gdrive_path='/content/gdrive/My Drive/Projects/17-0710_BBX/1_data/quality_analysis/fmriprep_files'

## BIDS


## Onsets 
original parser file: [.py](https://drive.google.com/drive/u/0/folders/1iD9uHpdnLXPPq9ORvynezd6-hWwBZfgH)



## Motion Check

### fMRIPREP   
path on renci: `/projects/niblab/experiments/bbx/data/fmriprep
`   


Here we are grab the 6 rigid-motion body parameters from the `.tsv` confound files generated from fMRIPrep for each subject & condition. 

In [ ]:
# variable lists to grab from the dataframe
moco_cols = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']
mot_outliers=['motion_outlier00', 'motion_outlier01', 'motion_outlier02',
              'motion_outlier03','motion_outlier04', 'motion_outlier05']

CONFOUND_TSVS=glob.glob(os.path.join(gdrive_path,"*confounds*.tsv"))
CONFOUND_TSVS.sort()
print("[INFO] %s tsvs found."%len(CONFOUND_TSVS))

[INFO] 689 tsvs found.


In [ ]:
subject_ids=list(set([x.split("/")[-1].split("_")[0] for x in CONFOUND_TSVS]))
subject_ids.sort()
print("[INFO] %s subjects found."%len(subject_ids))
print(subject_ids)

[INFO] 140 subjects found.
['sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017', 'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-028', 'sub-030', 'sub-031', 'sub-032', 'sub-033', 'sub-034', 'sub-035', 'sub-036', 'sub-037', 'sub-038', 'sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-045', 'sub-046', 'sub-048', 'sub-050', 'sub-051', 'sub-052', 'sub-053', 'sub-054', 'sub-055', 'sub-056', 'sub-057', 'sub-058', 'sub-059', 'sub-060', 'sub-061', 'sub-062', 'sub-063', 'sub-064', 'sub-065', 'sub-066', 'sub-067', 'sub-068', 'sub-070', 'sub-071', 'sub-072', 'sub-073', 'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-082', 'sub-083', 'sub-084', 'sub-085', 'sub-086', 'sub-087', 'sub-088', 'sub-089', 'sub-090', 'sub-091', 'sub-092', 'sub-093', 'sub-095', 'sub

In [ ]:
suspect_subjects={}


In [ ]:
for tsv in CONFOUND_TSVS[:10]:

  sub_id=tsv.split("/")[-1].split("_")[0]
  task=tsv.split("/")[-1].split("_")[2]
  #print('[INFO] %s'%subj_id)
  
  tsv_df=pd.read_csv(tsv, sep="\t")
  # get datafile column variables
  tsv_df.rename_axis("time(s)", inplace=True)

  # plot the 6 basic motion paramters
  #tsv_df[moco_cols].plot(kind='line', title="{}_{} six basic motion parameters".format(sub_id, task))
  
      
      
  min_pt = tsv_df['std_dvars'].min() - .3
  max_pt = tsv_df['std_dvars'].max() + .3
  max_time_pt = tsv_df.index.stop
  spikes=[]
  for index,pt in tsv_df['std_dvars'].iteritems():
      if pt > 3:
          spikes.append((round(pt,2), index))

  # print display only if spikes found
  if spike_ct > 0:
      
      #print("\n[INFO] {} {}  \n[INFO] Spike count: {}".format(sub_id, task,spike_ct))
      #print("[INFO] Spikes Found:")
      #for x in spikes:
        #   print("\tTime: {} \t| Spike: {}".format(x[1], x[0]) )
      if sub_id not in suspect_subjects:
        suspect_subjects[sub_id] = {}
      if task not in suspect_subjects:
        suspect_subjects[task] = spikes

In [ ]:
suspect_subjects

{}

In [ ]:
## Helper Functions


def display_motion():
  # display thte 6 rigid-body motion parameters, ""
  for tsv in CONFOUND_TSVS:

      sub_id=tsv.split("/")[-1].split("_")[0]
      task=tsv.split("/")[-1].split("_")[2]
      #print('[INFO] %s'%subj_id)
      
      tsv_df=pd.read_csv(tsv, sep="\t")
      # get datafile column variables
      tsv_df.rename_axis("time(s)", inplace=True)

      # plot the 6 basic motion paramters
      tsv_df[moco_cols].plot(kind='line', title="{}_{} six basic motion parameters".format(sub_id, task))
      plt.show()


def display_fd(suspect_subjects, CONFOUND_TSVS):

  for tsv in CONFOUND_TSVS:

      sub_id=tsv.split("/")[-1].split("_")[0]
      task=tsv.split("/")[-1].split("_")[2]
      #print('[INFO] %s'%subj_id)
      
      tsv_df=pd.read_csv(tsv, sep="\t")
      # get datafile column variables
      tsv_df.rename_axis("time(s)", inplace=True)

      # plot the 6 basic motion paramters
      #tsv_df[moco_cols].plot(kind='line', title="{}_{} six basic motion parameters".format(sub_id, task))
      
          
          
      min_pt = tsv_df['std_dvars'].min() - .3
      max_pt = tsv_df['std_dvars'].max() + .3
      max_time_pt = tsv_df.index.stop
      spikes=[]
      for index,pt in tsv_df['std_dvars'].iteritems():
          if pt > 3:
              spikes.append((round(pt,2), index))

      spike_ct=len(spikes)
      print(spike_ct)
      # print display only if spikes found
      if spike_ct > 0:
          
          #print("\n[INFO] {} {}  \n[INFO] Spike count: {}".format(sub_id, task,spike_ct))
          #print("[INFO] Spikes Found:")
          #for x in spikes:
           #   print("\tTime: {} \t| Spike: {}".format(x[1], x[0]) )
          if sub_id not in suspect_subjects:
            suspect_subjects[sub_id] = {}
          if task not in suspect_subjects:
            suspect_subjects[task] = spikes
      return suspect_subjects;
          
          #fig, ax = plt.subplots(ncols=2, figsize=(15,8))
          #tsv_df.plot.line( y="std_dvars", c="purple", use_index=True, ax=ax[0])
          #tsv_df.plot.line( y="framewise_displacement", use_index=True, ax=ax[1])
          #fig.legend(loc="best")

          #plt.show()
        
  

In [ ]:
suspect_subjects=display_fd(suspect_subjects, CONFOUND_TSVS)

0


In [ ]:
def plot_motion_params(tsv=tsv_df[0]):
  sub_id=tsv.split("/")[-1].split("_")[0]
  task=tsv.split("/")[-1].split("_")[2]
  #print('[INFO] %s'%subj_id)

  tsv_df=pd.read_csv(tsv, sep="\t")
  # get datafile column variables
  tsv_df.rename_axis("time(s)", inplace=True)

  # plot the 6 basic motion paramters
  tsv_df[moco_cols].plot(kind='line', title="{}_{} six basic motion parameters".format(sub_id, task))
  plt.show()


w = widgets.Select(options=T)
interactive(plot, x=w)

NameError: ignored

In [ ]:
def plot_motion_params(tsv=subject_ids[0]):
  sub_id=tsv.split("/")[-1].split("_")[0]
  task=tsv.split("/")[-1].split("_")[2]
  #print('[INFO] %s'%subj_id)

  tsv_df=pd.read_csv(tsv, sep="\t")
  # get datafile column variables
  tsv_df.rename_axis("time(s)", inplace=True)

  # plot the 6 basic motion paramters
  tsv_df[moco_cols].plot(kind='line', title="{}_{} six basic motion parameters".format(sub_id, task))
  plt.show()


w = widgets.Select(options=T)
interactive(plot, x=w)

#### Confounds

## FSL Feat 3 

In [ ]:
COPE_LIST=['c10_ssbtaste_h20taste_129',
 'c11_usbtaste_h20taste_129',
 'c13_h20taste_rinse_129',
 'c14_ssbcue_usbcue_129',
 'c15_usbcue_ssbcue_129',
 'c16_ssbtaste_usbtaste_129',
 'c17_usbtaste_ssbtaste_129',
 'c12_ssbtaste+usbtaste_h20taste_129',
 'c18_rinse_129',
 'c19_ssbtaste_rinse_129',
 'c1_ssbcue_129',
 'c20_usbtaste_rinse_129',
 'c2_usbcue_129',
 'c3_h20cue_129',
 'c4_ssbtaste_129',
 'c5_usbtaste_129',
 'c6_h20taste_129',
 'c7_ssbcue_h20cue_129',
 'c8_usbcue_h20cue_129',
 'c9_ssbcue+usbcue_h20cue_129']

In [ ]:
COPE_DICT={
  'c1_ssbcue_129': {"title": "cope 1, ssbcue > baseline" },
  'c2_usbcue_129': {"title": "cope 2, usbcue > baseline" },
  'c3_h20cue_129': {"title": "cope 3, h20cue > baseline" },
  'c4_ssbtaste_129': {"title": "cope 4, ssbtaste > baseline" },
  'c5_usbtaste_129': {"title": "cope 5, usbtaste > baseline"},
  'c6_h20taste_129': {"title": "cope 6, h20taste > baseline" },
  'c7_ssbcue_h20cue_129': {"title": "cope 7, ssbcue > h20cue" },
  'c8_usbcue_h20cue_129': {"title": "cope 8, usbcue > h20cue" },
  'c9_ssbcue+usbcue_h20cue_129': {"title": "cope 9, ssbcue + usbcue > h20cue"},
  'c10_ssbtaste_h20taste_129': {"title": "cope 10, ssbcue + usbcue > h20cue" },
  'c11_usbtaste_h20taste_129': {"title": "cope 11, usbtaste > h20taste" },
  'c12_ssbtaste+usbtaste_h20taste_129': { "title": "cope 12, ssbtaste + usbtaste > h20taste" },
  'c13_h20taste_rinse_129': {"title": "cope 13, h20taste > rinse" },
  'c14_ssbcue_usbcue_129': {"title": "cope 14, ssbcue > usbcue" },
  'c15_usbcue_ssbcue_129': {"title": "cope 15, usbcue > ssbcue" },
  'c16_ssbtaste_usbtaste_129': {"title": "cope 16, ssbtaste > usbtaste" } ,
  'c17_usbtaste_ssbtaste_129': {"title": "cope 17, usbtaste > ssbtaste" },
  'c18_rinse_129': {"title": "cope 18, rinse > baseline"},
  'c19_ssbtaste_rinse_129': {"title": "cope 19, ssbtaste > rinse"},
  'c20_usbtaste_rinse_129': {"title": "cope 20, usbtaste > rinse"},
  }

In [ ]:
data_dict={}

for cope in COPE_DICT.keys():
    folder_path='/content/gdrive/My Drive/Projects/17-0710_BBX/1_data/quality_analysis/files'
    cope_id = cope.split("_")[0]
    #print(cope)
    # get threshold png file for each cope
    zstat_img = os.path.join(folder_path, '%s_rendered_thresh_zstat1.png'%cope_id)
    #print("[INFO] zstat file path: %s"%zstat_img)
    
    cluster_txt = os.path.join(folder_path, '%s_cluster_zstat1_std.txt'%cope_id)
    #print("[INFO] cluster text file: %s"%cluster_txt)
    
    lmax_txt = os.path.join(folder_path, '%s_lmax_zstat1_std.txt'%cope_id)
    #print("[INFO] lmax text file: %s"%lmax_txt)
    
    if cope not in data_dict:
        data_dict[cope] = {}
    
    #data_dict[cope]['zstat_img'] = zstat_img
    #if "zstat_img" not in COPE_DICT[cope]
    COPE_DICT[cope]['zstat_img'] = zstat_img
    COPE_DICT[cope]['lmax_txt'] = lmax_txt
    COPE_DICT[cope]['cluster_txt'] = cluster_txt
    



In [ ]:
# check data dict was fill
COPE_DICT.keys()


dict_keys(['c1_ssbcue_129', 'c2_usbcue_129', 'c3_h20cue_129', 'c4_ssbtaste_129', 'c5_usbtaste_129', 'c6_h20taste_129', 'c7_ssbcue_h20cue_129', 'c8_usbcue_h20cue_129', 'c9_ssbcue+usbcue_h20cue_129', 'c10_ssbtaste_h20taste_129', 'c11_usbtaste_h20taste_129', 'c12_ssbtaste+usbtaste_h20taste_129', 'c13_h20taste_rinse_129', 'c14_ssbcue_usbcue_129', 'c15_usbcue_ssbcue_129', 'c16_ssbtaste_usbtaste_129', 'c17_usbtaste_ssbtaste_129', 'c18_rinse_129', 'c19_ssbtaste_rinse_129', 'c20_usbtaste_rinse_129'])

In [ ]:
menu_opts=[]
for x in COPE_DICT.keys():
  menu_opts.append(COPE_DICT[x]['title'])
menu_opts

['cope 1, ssbcue > baseline',
 'cope 2, usbcue > baseline',
 'cope 3, h20cue > baseline',
 'cope 4, ssbtaste > baseline',
 'cope 5, usbtaste > baseline',
 'cope 6, h20taste > baseline',
 'cope 7, ssbcue > h20cue',
 'cope 8, usbcue > h20cue',
 'cope 9, ssbcue + usbcue > h20cue',
 'cope 10, ssbcue + usbcue > h20cue',
 'cope 11, usbtaste > h20taste',
 'cope 12, ssbtaste + usbtaste > h20taste',
 'cope 13, h20taste > rinse',
 'cope 14, ssbcue > usbcue',
 'cope 15, usbcue > ssbcue',
 'cope 16, ssbtaste > usbtaste',
 'cope 17, usbtaste > ssbtaste',
 'cope 18, rinse > baseline',
 'cope 19, ssbtaste > rinse',
 'cope 20, usbtaste > rinse']

**FSL Feat 3 zstats from cope analysis**

Quick Widget Viewer:  
*To run...*


In [ ]:
## Helper Functions 

def get_key(x):

  if "cope 1," in x:
    return ''.join([x for x in COPE_LIST if "c1_" in x])
  elif "cope 2," in x:
    return ''.join([x for x in COPE_LIST if "c2_" in x])
  elif "cope 3," in x:
    return ''.join([x for x in COPE_LIST if "c3_" in x])
  elif "cope 4," in x:
    return ''.join([x for x in COPE_LIST if "c4_" in x])
  elif "cope 5," in x:
    return ''.join([x for x in COPE_LIST if "c5_" in x])
  elif "cope 6," in x:
    return ''.join([x for x in COPE_LIST if "c6_" in x])  
  elif "cope 7," in x:
    return ''.join([x for x in COPE_LIST if "c7_" in x])
    
  elif "cope 8," in x:
    return ''.join([x for x in COPE_LIST if "c8_" in x])
    
  elif "cope 9," in x:
    return ''.join([x for x in COPE_LIST if "c9_" in x])
    
  elif "cope 10," in x:
    return ''.join([x for x in COPE_LIST if "c10_" in x])
    
  elif "cope 11," in x:
    return ''.join([x for x in COPE_LIST if "c11_" in x])
    
  elif "cope 12," in x:
    return ''.join([x for x in COPE_LIST if "c12_" in x])
    
  elif "cope 13," in x:
    return ''.join([x for x in COPE_LIST if "c13_" in x])

  elif "cope 14," in x:
    return ''.join([x for x in COPE_LIST if "c14_" in x])
    
  elif "cope 15," in x:
    return ''.join([x for x in COPE_LIST if "c15_" in x])
    
  elif "cope 16," in x:
    return ''.join([x for x in COPE_LIST if "c16_" in x])
    
  elif "cope 17," in x:
    return ''.join([x for x in COPE_LIST if "c17_" in x])
    
  elif "cope 18," in x:
    return ''.join([x for x in COPE_LIST if "c18_" in x])
    
  elif "cope 19," in x:
    return ''.join([x for x in COPE_LIST if "c19_" in x])
    
  elif "cope 20," in x:
    return ''.join([x for x in COPE_LIST if "c20_" in x])
    
  else:
    return None

  return key;


def plot(cope=menu_opts[0]):
    plt.rcParams["figure.figsize"] = (30,15)

    # get cope key 
    key = get_key(cope)
    folder_renci="/projects/niblab/experiments/bbx/data/fsl_feat3/ses-1/%s.gfeat"%key


    # with key, get files from the dictionary
    img=COPE_DICT[key]['zstat_img']
    img_name=img.split("/")[-1]
    img_path=os.path.join(folder_renci, 'cope1.feat')
    lmax_txt=COPE_DICT[key]['lmax_txt']
    cluster_txt = COPE_DICT[key]['cluster_txt']
    
    # display cope title
    print('\n\n[INFO] %s '%COPE_DICT[key]['title'])
    print('[INFO] folder name: %s.gfeat '%key)
    print("[INFO] find folder, images and other files on renci: \n\t%s \n"%img_path)  


    # display image

    print("[INFO] image below is the zstat threshold rendered image (%s)"%img_name)
    img = mpimg.imread(img)
    plt.imshow(img)
    plt.show()


    # display cluster z stats
    print("\n[FILE] stats below from file %s"%cluster_txt.split("/")[-1])
    cluster_data=pd.read_csv(cluster_txt, sep="\t")
    display(cluster_data)

    # display lmax z stats  
    print("\n\n[FILE] stats below from file %s"%lmax_txt.split("/")[-1])
    lmax_data=pd.read_csv(lmax_txt, sep="\t")
    del lmax_data['Unnamed: 5']
    display(lmax_data)



In [ ]:
# run widget that calls help functions above
w = widgets.Select(options=menu_opts)
interactive(plot, cope=w)

interactive(children=(Select(description='cope', options=('cope 1, ssbcue > baseline', 'cope 2, usbcue > basel…